# Single State Search
## Solving Knapsack problem

### Imports and Declaration

In [1]:
import random
import copy
import math
import queue

In [2]:
def generateVals(length, vmax, wmax):
    gbValues = []
    gbWeights = []

    while len(gbValues) < length:
        gbValues.append(random.randint(1,vmax))
        gbWeights.append(random.randint(1,wmax))
        
    return gbValues, gbWeights

In [3]:
'''
Declared global variables for the knapsack problem
'''
gValues, gWeights = generateVals(50, 75, 50)
fullWeight = 500

In [4]:
print("List of values of items: ", gValues)
print("List of weights of items: ", gWeights)

List of values of items:  [29, 56, 62, 50, 25, 48, 60, 57, 17, 17, 51, 52, 15, 58, 46, 31, 75, 66, 4, 57, 35, 43, 4, 11, 68, 47, 43, 1, 72, 25, 3, 38, 44, 28, 55, 22, 8, 21, 48, 45, 8, 58, 45, 37, 71, 39, 9, 15, 61, 19]
List of weights of items:  [40, 41, 12, 45, 39, 9, 13, 37, 16, 37, 1, 36, 27, 2, 47, 45, 26, 22, 28, 25, 16, 50, 6, 41, 14, 14, 40, 23, 30, 13, 31, 20, 50, 46, 30, 15, 44, 34, 41, 2, 5, 46, 21, 37, 15, 33, 50, 1, 48, 47]


## Generic functions used in different methods

In [5]:
'''
Create random solution
input: s <-- size of the list
output: [0,1,1,0,0,1]
'''
def randomSolution(s):
    temp = []
    for i in range(s):
        temp.append(random.randrange(2))
        
    return temp

In [6]:
'''
Retrive the total weight and total value of the provided solution
input: solution <-- bit-list solution, identifies which items to include in the knapsack
output: total weight, total value
'''
def getWV(solu):
    tWeight = 0
    tValue = 0
    for i in range(len(solu)):
        if solu[i] == 1:
            tWeight += gWeights[i]
            tValue += gValues[i]
            
    return tWeight, tValue

In [7]:
'''
Function to determine if the solution fits the constraint for the knapsack problem
input: solution <-- bit-list solution, identifies which items to include in the knapsack
output: 0 or 1
'''
def fitnessW(solu):
    tempW, tempV = getWV(solu)
    if tempW > fullWeight:
        return 0
    return 1

In [8]:
'''
Function to flip the last bit of the solution
input: solution <-- bit-list solution, identifies which items to include in the knapsack
output: solution
'''
def tweak(sol):
    carry = False
    for i in reversed(range(len(sol))):
        if carry:
            if sol[i] > 0:
                sol[i] = 0
                carry = True
            else:
                sol[i] = 1
                carry = False
        else:
            if sol[i] > 0:
                sol[i] = 0
                carry = True
            else:
                sol[i] = 1
                carry = False
                break
                
    return sol

## _**Local Search**_

#### _Fitness Function_

In [9]:
'''
No exact fitness calculated but on the comparison of the two solutions.
SolutionY is made sure to have passed the constraint for the weight
before it can be included in the function
input: solutionX, and solutionY <-- bit-list solution, identifies which items to include in the knapsack
output: solutionX or solutionY
'''
def fitnessLS(solX, solY):
    xWeight, xValue = getWV(solX)
    yWeight, yValue = getWV(solY)
    if xWeight > fullWeight:
        return solY
        
    else:
        if xValue > yValue:
            return solX
        else:
            return solY

#### _Running Local Search_

In [10]:
'''
Function to flip the last bit of the solution
input: size <-- size of the neighborhood
        x <-- solution to check neighbor with
        best <-- best solution found, will be replaced if there is more fitting solution
output: best
'''
def checkNeighbor(size, x, best):
    y = copy.deepcopy(x)
    for i in range(size):
        y = tweak(y)
        best = fitnessLS(y, best)
            
    return best

In [11]:
def localSearch():
    proceed = True
    stopping = 30
    ngbSize = 20
    ranX = randomSolution(len(gValues))
    while fitnessW(ranX) == 0:
        ranX = randomSolution(len(gValues))

    bestX = copy.deepcopy(ranX)
    bestX = tweak(bestX)
    while fitnessW(bestX) == 0:
        bestX = tweak(bestX)
        
    while(proceed and stopping > 0):
        bestX = checkNeighbor(ngbSize, ranX, bestX)
        xW, xV = getWV(ranX)
        bW, bV = getWV(bestX)
        if xV < bV:
            xV = copy.deepcopy(bV)
            proceed = True
            stopping = stopping - 1
        else:
            proceed = False
            
    print("Optimal Solution: ")
    for i in range(len(gValues) - 1):
        if ranX[i] == 1:
            print("Item: {}, Value: {}, Weight: {}".format(i,gValues[i], gWeights[i]))
            
    print("Total Weight and Value: {}".format(getWV(ranX)))

In [12]:
%%time
localSearch()

Optimal Solution: 
Item: 0, Value: 29, Weight: 40
Item: 2, Value: 62, Weight: 12
Item: 3, Value: 50, Weight: 45
Item: 5, Value: 48, Weight: 9
Item: 6, Value: 60, Weight: 13
Item: 11, Value: 52, Weight: 36
Item: 13, Value: 58, Weight: 2
Item: 14, Value: 46, Weight: 47
Item: 18, Value: 4, Weight: 28
Item: 20, Value: 35, Weight: 16
Item: 22, Value: 4, Weight: 6
Item: 30, Value: 3, Weight: 31
Item: 32, Value: 44, Weight: 50
Item: 34, Value: 55, Weight: 30
Item: 36, Value: 8, Weight: 44
Item: 37, Value: 21, Weight: 34
Item: 40, Value: 8, Weight: 5
Item: 46, Value: 9, Weight: 50
Total Weight and Value: (498, 596)
Wall time: 16 ms


## Simulated Annealing

#### _Fitness Function_

In [13]:
'''
Template provided by Essentials of Metaheuristic
Upper bound probability to consider whether to accept solution that is less than
the solution being compared to
input: t <-- temperature stated for the method
        xV <-- total value of solution X
        yV <-- total value of solution Y
output: probability
'''
def probAccept(t, xV, yV):
    return math.e**((xV - yV) / t)

'''
Template provided by Essentials of Metaheuristic
Fitness function is similar to local search but with probability of accepting solution that
is less than the solution being compared to
input: t <-- temperature to include in setting probability for accepting solution less than the best
        solutionX, and solutionY <-- bit-list solution, identifies which items to include in the knapsack
output: solutionX or solutionY
'''
def fitnessSA(t, solX, solY):
    xWeight, xValue = getWV(solX)
    yWeight, yValue = getWV(solY)
    if xWeight > fullWeight:
        return False
        
    else:
        if xValue > yValue or random.uniform(0.0,1.0) < probAccept(t,xValue, yValue):
            return True
        else:
            return False

#### _Running Simulated Annealing_

In [14]:
def simulatedAnnealing():
    temperature = 100
    tempDec = 1
    
    # get random solution until fits criteria
    ranXSA = randomSolution(len(gValues))
    while fitnessW(ranXSA) == 0:
        ranXSA = randomSolution(len(gValues))

    # tweak initial solution until fits criteria
    bestXSA = copy.deepcopy(ranXSA)
        
    # run SA
    while(temperature > 0):
        rsa = tweak(copy.deepcopy(ranXSA))
        while fitnessW(rsa) == 0:
            rsa = tweak(rsa)
            
        if fitnessSA(temperature, rsa, ranXSA):
            ranXSA = rsa
            
        temperature = temperature - tempDec
        bestW, bestV = getWV(bestXSA)
        sW, sV = getWV(ranXSA)
        if sV > bestV:
            bestXSA = copy.deepcopy(ranXSA)
            
    
    # display optimal solution
    print("Optimal Solution: ")
    for i in range(len(gValues) - 1):
        if ranXSA[i] == 1:
            print("Item: {}, Value: {}, Weight: {}".format(i,gValues[i], gWeights[i]))
            
    print("Total Weight and Value: {}".format(getWV(ranXSA)))

In [15]:
%%time
simulatedAnnealing()

Optimal Solution: 
Item: 2, Value: 62, Weight: 12
Item: 3, Value: 50, Weight: 45
Item: 4, Value: 25, Weight: 39
Item: 5, Value: 48, Weight: 9
Item: 11, Value: 52, Weight: 36
Item: 13, Value: 58, Weight: 2
Item: 14, Value: 46, Weight: 47
Item: 17, Value: 66, Weight: 22
Item: 18, Value: 4, Weight: 28
Item: 20, Value: 35, Weight: 16
Item: 21, Value: 43, Weight: 50
Item: 24, Value: 68, Weight: 14
Item: 38, Value: 48, Weight: 41
Item: 39, Value: 45, Weight: 2
Item: 40, Value: 8, Weight: 5
Item: 42, Value: 45, Weight: 21
Item: 43, Value: 37, Weight: 37
Item: 44, Value: 71, Weight: 15
Item: 46, Value: 9, Weight: 50
Total Weight and Value: (491, 820)
Wall time: 43.9 ms


## _**Tabu Search**_

#### _Fitness Function_

In [16]:
'''
Template provided by Essentials of Metaheuristic
Fitness function is similar to local search
input: solutionX, and solutionY <-- bit-list solution, identifies which items to include in the knapsack
output: solutionX or solutionY
'''
def fitnessTS(solX, solY):
    xWeight, xValue = getWV(solX)
    yWeight, yValue = getWV(solY)
    if xWeight > fullWeight:
        return False
        
    else:
        if xValue > yValue:
            return solX
        else:
            return solY

#### _Running Tabu Search_

In [17]:
def tabuSearch():
    # initialize tabu list
    iterCount = 0
    tbSize = len(gValues) 
    tbList = queue.Queue()
    nTweaks = 20
    
    # get random solution until fits criteria
    ranXTS = randomSolution(len(gValues))
    while fitnessW(ranXTS) == 0:
        ranXTS = randomSolution(len(gValues))

    # add initial solution to tabu list
    bestXTS = copy.deepcopy(ranXTS)
    tbList.put(ranXTS)
        
    # run tabu search
    while(iterCount >= 15):
        if tblist.qsize() > tbSize:
            tblist.get()
            
        rts = tweak(copy.deepcopy(ranXTS))

        for i in range(nTweaks - 1):
            wts = tweak(copy.deepcopy(ranXTS))
            if fitnessTS(rts, wts):
                rts = copy.deepcopy(wts)
                
        if rts not in tbList:
            ranXTS = copy.deepcopy(rts)
            tblist.put(rts)
        if fitnessTS(ranXTS, bestXTS):
            bestXTS = copy.deepcopy(ranXTS)
            iterCount = 0
        else:
            iterCount += 1
        
    # display optimal solution
    print("Optimal Solution: ")
    for i in range(len(gValues) - 1):
        if ranXTS[i] == 1:
            print("Item: {}, Value: {}, Weight: {}".format(i,gValues[i], gWeights[i]))
            
    print("Total Weight and Value: {}".format(getWV(ranXTS)))

In [18]:
%%time
tabuSearch()

Optimal Solution: 
Item: 8, Value: 17, Weight: 16
Item: 9, Value: 17, Weight: 37
Item: 10, Value: 51, Weight: 1
Item: 11, Value: 52, Weight: 36
Item: 12, Value: 15, Weight: 27
Item: 13, Value: 58, Weight: 2
Item: 14, Value: 46, Weight: 47
Item: 16, Value: 75, Weight: 26
Item: 19, Value: 57, Weight: 25
Item: 24, Value: 68, Weight: 14
Item: 26, Value: 43, Weight: 40
Item: 32, Value: 44, Weight: 50
Item: 36, Value: 8, Weight: 44
Item: 37, Value: 21, Weight: 34
Item: 42, Value: 45, Weight: 21
Item: 45, Value: 39, Weight: 33
Total Weight and Value: (453, 656)
Wall time: 0 ns
